# Eval

We are going to excecute a set of tests over a ser of excercises , get the improved code and save a historical in a dataframe.

Excercise 1 : Fibonacci

In [1]:
import unittest
import sys
sys.path.append('src/excercise1-fibonacci')
from fibonacci_test import TestFibonacci

# Ejecutar todas las pruebas
unittest.main(argv=['first-arg-is-ignored'], exit=False)

# O ejecutar pruebas específicas
suite = unittest.TestSuite()
suite.addTest(TestFibonacci('test_empty_sequence'))
runner = unittest.TextTestRunner()
runner.run(suite)


.........
----------------------------------------------------------------------
Ran 9 tests in 0.002s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


Empezando el calculo de algo...
No se hace nada jeje
Empezando el calculo de algo...
Resultado (no optimizado): [0, 1, 1, 2, 3]
Empezando el calculo de algo...
Empezando el calculo de algo...
Resultado (no optimizado): [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]
Empezando el calculo de algo...
Empezando el calculo de algo...
Resultado (no optimizado): [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
Empezando el calculo de algo...
Resultado (no optimizado): [0, 1]
Empezando el calculo de algo...
Resultado (no optimizado): [0, 1, 1, 2, 3]
Empezando el calculo de algo...
Resultado (no optimizado): [5, 10, 0, 1]
Empezando el calculo de algo...


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

# Individual Test Excecution

In [2]:
import unittest
import pandas as pd
import sys
import time
from io import StringIO
import contextlib

# Añadir la ruta al directorio que contiene los archivos de prueba
sys.path.append('src/excercise1-fibonacci')
from fibonacci_test import TestFibonacci

# Clase personalizada para capturar resultados de pruebas
class TestResultCollector(unittest.TextTestResult):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.test_results = []
    
    def addSuccess(self, test):
        super().addSuccess(test)
        self.test_results.append({
            'test_name': test.id().split('.')[-1],
            'status': 'PASS',
            'execution_time': getattr(test, 'execution_time', 0),
            'error_message': None
        })
    
    def addFailure(self, test, err):
        super().addFailure(test, err)
        self.test_results.append({
            'test_name': test.id().split('.')[-1],
            'status': 'FAIL',
            'execution_time': getattr(test, 'execution_time', 0),
            'error_message': str(err[1])
        })
    
    def addError(self, test, err):
        super().addError(test, err)
        self.test_results.append({
            'test_name': test.id().split('.')[-1],
            'status': 'ERROR',
            'execution_time': getattr(test, 'execution_time', 0),
            'error_message': str(err[1])
        })

class TimedTestCase(unittest.TestCase):
    def run(self, result=None):
        start_time = time.time()
        super().run(result)
        self.execution_time = time.time() - start_time

class TimedTestFibonacci(TestFibonacci, TimedTestCase):
    pass

suite = unittest.TestLoader().loadTestsFromTestCase(TimedTestFibonacci)

output = StringIO()
with contextlib.redirect_stdout(output):
    runner = unittest.TextTestRunner(resultclass=TestResultCollector)
    result = runner.run(suite)

test_results_df = pd.DataFrame(result.test_results)

total_tests = len(test_results_df)
passed_tests = len(test_results_df[test_results_df['status'] == 'PASS'])
success_rate = (passed_tests / total_tests) * 100 if total_tests > 0 else 0

summary = pd.DataFrame([{
    'test_name': 'SUMMARY',
    'status': f'{passed_tests}/{total_tests} ({success_rate:.2f}%)',
    'execution_time': test_results_df['execution_time'].sum(),
    'error_message': None
}])

# Combinar resultados y resumen
results_df = pd.concat([test_results_df, summary], ignore_index=True)

# Mostrar el DataFrame
display(results_df)

# Mostrar solo el porcentaje de éxito
print(f"\nPorcentaje de éxito: {success_rate:.2f}%")


.........
----------------------------------------------------------------------
Ran 9 tests in 0.002s

OK


,test_name,status,execution_time,error_message
0,test_c_parameter_false,PASS,0,None
1,test_custom_start_values,PASS,0,None
2,test_empty_sequence,PASS,0,None
3,test_large_sequence,PASS,0,None
4,test_single_element,PASS,0,None
5,test_standard_fibonacci,PASS,0,None
6,test_two_elements,PASS,0,None
7,test_with_args_kwargs,PASS,0,None
8,test_with_provided_x,PASS,0,None
9,SUMMARY,9/9 (100.00%),0,None



Porcentaje de éxito: 100.00%


# Test Excecution

In [3]:
import unittest
import pandas as pd
import sys
import time
import importlib
import os
from io import StringIO
import contextlib
import inspect

def run_tests_for_file(file_path, iteration=1):
    """
    Ejecuta las pruebas para un archivo específico y devuelve los resultados.
    
    Args:
        file_path: Ruta al archivo de prueba (sin extensión .py)
        iteration: Número de iteración
        
    Returns:
        DataFrame con los resultados
    """
    file_dir = os.path.dirname(file_path)
    file_name = os.path.basename(file_path)
    
    if file_dir and file_dir not in sys.path:
        sys.path.append(file_dir)
    
    try:
        test_module = importlib.import_module(file_name)
        
        test_classes = []
        for name, obj in inspect.getmembers(test_module):
            if inspect.isclass(obj) and issubclass(obj, unittest.TestCase) and obj != unittest.TestCase:
                test_classes.append(obj)
        
        if not test_classes:
            return pd.DataFrame({
                'file': [file_path],
                'iteration': [iteration],
                'tests': ['0/0'],
                'percentage_of_success': [0.0],
                'execution_time': [0.0]
            })
        
        all_results = []
        total_passed = 0
        total_tests = 0
        total_time = 0
        
        for test_class in test_classes:
            class TimedTestCase(test_class):
                def run(self, result=None):
                    start_time = time.time()
                    super().run(result)
                    self.execution_time = time.time() - start_time
            
            class TestResultCollector(unittest.TextTestResult):
                def __init__(self, *args, **kwargs):
                    super().__init__(*args, **kwargs)
                    self.test_results = []
                
                def addSuccess(self, test):
                    super().addSuccess(test)
                    self.test_results.append({
                        'test_name': test.id().split('.')[-1],
                        'status': 'PASS',
                        'execution_time': getattr(test, 'execution_time', 0),
                        'error_message': None
                    })
                
                def addFailure(self, test, err):
                    super().addFailure(test, err)
                    self.test_results.append({
                        'test_name': test.id().split('.')[-1],
                        'status': 'FAIL',
                        'execution_time': getattr(test, 'execution_time', 0),
                        'error_message': str(err[1])
                    })
                
                def addError(self, test, err):
                    super().addError(test, err)
                    self.test_results.append({
                        'test_name': test.id().split('.')[-1],
                        'status': 'ERROR',
                        'execution_time': getattr(test, 'execution_time', 0),
                        'error_message': str(err[1])
                    })
            
            suite = unittest.TestLoader().loadTestsFromTestCase(TimedTestCase)
            
            output = StringIO()
            with contextlib.redirect_stdout(output):
                runner = unittest.TextTestRunner(resultclass=TestResultCollector)
                result = runner.run(suite)
            
            class_results = result.test_results
            all_results.extend(class_results)
            
            class_passed = len([r for r in class_results if r['status'] == 'PASS'])
            class_total = len(class_results)
            class_time = sum(r['execution_time'] for r in class_results)
            
            total_passed += class_passed
            total_tests += class_total
            total_time += class_time
        
        success_percentage = (total_passed / total_tests * 100) if total_tests > 0 else 0
        
        summary_df = pd.DataFrame({
            'file': [file_path],
            'iteration': [iteration],
            'tests': [f"{total_passed}/{total_tests}"],
            'percentage_of_success': [success_percentage],
            'execution_time': [total_time]
        })
        
        return summary_df
        
    except Exception as e:
        return pd.DataFrame({
            'file': [file_path],
            'iteration': [iteration],
            'tests': ['ERROR'],
            'percentage_of_success': [0.0],
            'execution_time': [0.0],
            'error': [str(e)]
        })

def run_tests_for_multiple_files(file_paths, iterations=1):
    """
    Ejecuta pruebas para múltiples archivos y devuelve un DataFrame consolidado.
    
    Args:
        file_paths: Lista de rutas a archivos de prueba (sin extensión .py)
        iterations: Número de iteraciones para cada archivo
        
    Returns:
        DataFrame consolidado con todos los resultados
    """
    all_results = []
    
    for file_path in file_paths:
        for i in range(1, iterations + 1):
            result_df = run_tests_for_file(file_path, i)
            all_results.append(result_df)
    
    if all_results:
        return pd.concat(all_results, ignore_index=True)
    else:
        return pd.DataFrame(columns=['file', 'iteration', 'tests', 'percentage_of_success', 'execution_time'])


test_files = ['src/excercise1-fibonacci/fibonacci_test']  
iterations = 3  

results_df = run_tests_for_multiple_files(test_files, iterations)

display(results_df)

# Opcional: guardar resultados en CSV
# results_df.to_csv('test_results.csv', index=False)


.........
----------------------------------------------------------------------
Ran 9 tests in 0.002s

OK
.........
----------------------------------------------------------------------
Ran 9 tests in 0.002s

OK
.........
----------------------------------------------------------------------
Ran 9 tests in 0.002s

OK


,file,iteration,tests,percentage_of_success,execution_time
0,src/excercise1-fibonacci/fibonacci_test,1,9/9,100.0,0
1,src/excercise1-fibonacci/fibonacci_test,2,9/9,100.0,0
2,src/excercise1-fibonacci/fibonacci_test,3,9/9,100.0,0
